Importing libs

In [11]:
import gpxpy
import gpxpy.gpx
import pandas as pd
import pytz
from pytz import all_timezones

loading gpx file

In [12]:
gpx_file = "danielle GPX .GPX"
with open(gpx_file, "r") as file:
    gpx = gpxpy.parse(file)

In [13]:
gpx

GPX(tracks=[GPXTrack(segments=[GPXTrackSegment(points=[...])])])

In [38]:
gpx.tracks[0].segments[0].points[:10]

[GPXTrackPoint(34.021526, -118.288752, elevation=90.12919518072158),
 GPXTrackPoint(34.021566, -118.28889, elevation=91.4642141405493),
 GPXTrackPoint(34.02147, -118.288567, elevation=92.40057468786836),
 GPXTrackPoint(34.021553, -118.288315, elevation=91.63135995343328),
 GPXTrackPoint(34.021452, -118.288464, elevation=91.29162885062397),
 GPXTrackPoint(34.021578, -118.288401, elevation=91.90533726289868),
 GPXTrackPoint(34.021503, -118.288488, elevation=91.72100736759603),
 GPXTrackPoint(34.021594, -118.288463, elevation=91.78886562027037),
 GPXTrackPoint(34.02156, -118.288531, elevation=93.04319773428142),
 GPXTrackPoint(34.021578, -118.288662, elevation=93.90022918581963)]

In [17]:
import xml.etree.ElementTree as ET
import csv
from datetime import datetime


# Function to parse custom time format
def parse_custom_time(time_str):
    dt = datetime.strptime(time_str, "%m/%d/%Y, %I:%M:%S %p")
    return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M:%S")

Data Extraction from gpx

In [18]:
gpx_file_path = "danielle GPX .GPX"
tree = ET.parse(gpx_file_path)
root = tree.getroot()

# Define the namespaces
namespaces = {"default": "http://www.topografix.com/GPX/1/1"}

# Extract data from GPX
data = []
for trkpt in root.findall(".//default:trkpt", namespaces):
    lat = trkpt.get("lat")
    lon = trkpt.get("lon")
    ele = trkpt.find("default:ele", namespaces).text
    time = trkpt.find("default:time", namespaces).text
    date, time = parse_custom_time(time)  # Convert time to custom format
    data.append(
        {
            "latitude": lat,
            "longitude": lon,
            "altitude (m)": ele,
            "date": date,
            "time": time,
        }
    )

In [159]:
data[:2]

[{'latitude': '34.021526',
  'longitude': '-118.288752',
  'altitude (m)': '90.12919518072158',
  'date': '2024-04-16',
  'time': '15:55:47'},
 {'latitude': '34.021566',
  'longitude': '-118.288890',
  'altitude (m)': '91.4642141405493',
  'date': '2024-04-16',
  'time': '15:55:50'}]

In [21]:
gpx_data = pd.DataFrame(data)
gpx_data.head()

,latitude,longitude,altitude (m),date,time
0,34.021526,-118.288752,90.12919518072158,2024-04-16,15:55:47
1,34.021566,-118.288890,91.4642141405493,2024-04-16,15:55:50
2,34.021470,-118.288567,92.40057468786836,2024-04-16,15:55:54
3,34.021553,-118.288315,91.63135995343328,2024-04-16,15:56:01
4,34.021452,-118.288464,91.29162885062397,2024-04-16,15:56:26


Import GPX Data to CSV

In [161]:
gpx_data.to_csv("danielle.csv", index=False)

https://gist.github.com/pianosnake/b4a45ef6bgpx_data2ffb2e1b44bbcca107298

In [22]:
import math

EARTH_CIR_METERS = 40075016.686
degreesPerMeter = 360 / EARTH_CIR_METERS


def toRadians(degrees):
    return degrees * math.pi / 180


def latLngToBounds(lat, lng, zoom, width, height):
    metersPerPixelEW = EARTH_CIR_METERS / math.pow(2, zoom + 8)
    metersPerPixelNS = (
        EARTH_CIR_METERS / math.pow(2, zoom + 8) * math.cos(toRadians(lat))
    )

    shiftMetersEW = width / 2 * metersPerPixelEW
    shiftMetersNS = height / 2 * metersPerPixelNS

    shiftDegreesEW = shiftMetersEW * degreesPerMeter
    shiftDegreesNS = shiftMetersNS * degreesPerMeter

    minX = lng - shiftDegreesEW
    minY = lat - shiftDegreesNS
    maxX = lng + shiftDegreesEW
    maxY = lat + shiftDegreesNS

    return f"{minX:.4f},{minY:.6f},{maxX:.4f},{maxY:.6f}"

In [23]:
# convert data to boundary box

gpx_data["latitude"] = gpx_data["latitude"].astype(float)
gpx_data["longitude"] = gpx_data["longitude"].astype(float)

gpx_data["bbox"] = gpx_data.apply(
    lambda x: latLngToBounds(x["latitude"], x["longitude"], 12, 400, 400), axis=1
)

In [267]:
gpx_data.head()

,latitude,longitude,altitude (m),date,time,bbox
0,34.021526,-118.288752,90.12919518072158,2024-04-16,15:55:47,"-118.3574,33.964615,-118.2201,34.078437"
1,34.021566,-118.288890,91.4642141405493,2024-04-16,15:55:50,"-118.3576,33.964655,-118.2202,34.078477"
2,34.021470,-118.288567,92.40057468786836,2024-04-16,15:55:54,"-118.3572,33.964559,-118.2199,34.078381"
3,34.021553,-118.288315,91.63135995343328,2024-04-16,15:56:01,"-118.3570,33.964642,-118.2197,34.078464"
4,34.021452,-118.288464,91.29162885062397,2024-04-16,15:56:26,"-118.3571,33.964541,-118.2198,34.078363"


In [24]:
import requests
import csv
from io import StringIO


def get_air_quality(gpx_data):
    # Extract the hour from the time
    hour = gpx_data["time"].split(":")[0]
    header_array = [
        "Latitude",
        "Longitude",
        "UTC",
        "Parameter",
        "Unit",
        "AQI",
        "Category",
    ]
    # Construct the URL with the date, bbox, and time from gpx_data
    url = f"https://www.airnowapi.org/aq/data/?startDate={gpx_data['date']}T{hour}&endDate={gpx_data['date']}T{hour}&parameters=PM25,PM10&BBOX={gpx_data['bbox']}&dataType=A&format=text/csv&verbose=0&monitorType=0&includerawconcentrations=0&API_KEY=342FB14E-3637-470D-BEAE-A5DF1E193ADB"
    # url = f"https://www.airnowapi.org/aq/data/?startDate=2024-06-21T17&endDate=2024-06-21T18&parameters=PM25,PM10&BBOX=-124.205070,28.716781,-75.337882,45.419415&dataType=A&format=text/csv&verbose=0&monitorType=0&includerawconcentrations=0&API_KEY=342FB14E-3637-470D-BEAE-A5DF1E193ADB"
    # print(url)
    response = requests.get(url)
    return response.text

In [295]:
import pandas as pd
from io import StringIO

results = []
for index, row in gpx_data.head(400).iterrows():
    results.append(get_air_quality(row))

# Join the results into a single string
results_str = "\n".join(results)

data_io = StringIO(results_str)
df_temp = pd.read_csv(data_io, header=None)

# Set the column names
df_temp.columns = [
    "Latitude",
    "Longitude",
    "UTC",
    "Parameter",
    "AQI",
    "Category",
]

# Append the data to the final dataframe

df_temp.head()
df_temp.to_csv("air_quality.csv", index=False)

In [297]:
# Continue processing rows 401 to 700
for index, row in gpx_data.iloc[400:700].iterrows():
    results.append(get_air_quality(row))

# Join the results into a single string
results_str = "\n".join(results)

data_io = StringIO(results_str)
df_temp = pd.read_csv(data_io, header=None)

# Set the column names
df_temp.columns = [
    "Latitude",
    "Longitude",
    "UTC",
    "Parameter",
    "AQI",
    "Category",
]

# Append the data to the existing csv file
with open("air_quality.csv", "a") as f:
    df_temp.to_csv(f, header=False, index=False)

In [25]:
sample_data = [
    {
        "latitude": 34.019451,
        "longitude": -118.289409,
        "date": "2024-04-08",
        "time": "8:00:00",
    },
    {
        "latitude": 34.024496,
        "longitude": -118.278309,
        "date": "2024-04-11",
        "time": "8:00:00",
    },
    {
        "latitude": 34.025839,
        "longitude": -118.27746,
        "date": "2024-04-13",
        "time": "8:00:00",
    },
    {
        "latitude": 34.019611,
        "longitude": -118.289287,
        "date": "2024-04-15",
        "time": "8:00:00",
    },
    {
        "latitude": 34.01943,
        "longitude": -118.289417,
        "date": "2024-04-17",
        "time": "8:00:00",
    },
    {
        "latitude": 34.024762,
        "longitude": -118.279302,
        "date": "2024-04-19",
        "time": "8:00:00",
    },
]


sample_data_df = pd.DataFrame(sample_data)
sample_data_df["bbox"] = sample_data_df.apply(
    lambda x: latLngToBounds(x["latitude"], x["longitude"], 12, 400, 400), axis=1
)

sample_data_df.head()

results = []
for index, row in sample_data_df.iterrows():
    results.append(get_air_quality(row))

# Join the results into a single string

results_str = "\n".join(results)

data_io = StringIO(results_str)

df_temp = pd.read_csv(data_io, header=None)

# Set the column names
df_temp.columns = [
    "Latitude",
    "Longitude",
    "UTC",
    "Parameter",
    "AQI",
    "Category",
]

df_temp.head()

with open("air_quality.csv", "a") as f:
    df_temp.to_csv(f, header=False, index=False)

data extraction from survey data

In [4]:
import pandas as pd
import os

directory = "surveyData"

dataframes = {}

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        dataframes[filename] = pd.read_csv(os.path.join(directory, filename))

In [6]:
dataframes.keys()

dict_keys(['Ping1.csv', 'Ping2.csv', 'Ping3.csv', 'Ping6.csv', 'Ping4.csv', 'Ping5.csv'])

In [8]:
ping1 = dataframes["Ping1.csv"]
ping2 = dataframes["Ping2.csv"]
ping3 = dataframes["Ping3.csv"]
ping4 = dataframes["Ping4.csv"]
ping5 = dataframes["Ping5.csv"]
ping6 = dataframes["Ping6.csv"]

In [9]:
ping1.head()

,mbl_cod,rsp_id,instance_id,scheduled_start_local,timezone_offset,actual_start_local,SLEEP_9379186,HAPPY_JOYFUL_9379200,ENERGETIC_9379201,IRRITABLE_ANGRY_9379202,...,RESPIRATION,BODY_BATTERY,STEPS,CALORIES,FLOORS,INTENSITY_MINUTES,LONGITUDE,LATITUDE,AVG_AMP,VOX_ACTV
0,27181246,50695,1712592000,2024-04-08 09:00:00,-420,2024-04-08 09:34:46,3,7.0,6.0,6.0,...,0.0,0.0,0,0,0,0,0.000000,0.000000,0.0,0.0
1,27181246,50695,1712678400,2024-04-09 09:00:00,-420,2024-04-09 09:06:19,3,7.0,6.0,5.0,...,0.0,0.0,0,0,0,0,0.000000,0.000000,0.0,0.0
2,27181246,50695,1712764800,2024-04-10 09:00:00,-420,2024-04-10 09:33:15,4,7.0,6.0,5.0,...,0.0,0.0,0,0,0,0,34.019451,-118.289409,0.0,0.0
3,27181246,50695,1712851200,2024-04-11 09:00:00,-420,2024-04-11 09:00:05,3,1.0,2.0,3.0,...,0.0,0.0,0,0,0,0,34.024496,-118.278309,0.0,0.0
4,27181246,50695,1712937600,2024-04-12 09:00:00,-420,2024-04-12 09:04:59,3,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0,0.000000,0.000000,0.0,0.0


In [56]:
from datetime import datetime, timedelta

ping1["bbox"] = ping1.apply(
    lambda x: latLngToBounds(x["LONGITUDE"], x["LATITUDE"], 12, 400, 400), axis=1
)


def isInsideBbox(lat, lng, bbox):
    lng = float(lng)
    lat = float(lat)
    bbox = bbox.split(",")
    if (
        lng >= float(bbox[0])
        and lng <= float(bbox[2])
        and lat >= float(bbox[1])
        and lat <= float(bbox[3])
    ):
        return True
    return False


results = []


# filter the air quality from air_quality.csv that is inside the bounding box

air_quality = pd.read_csv("air_quality.csv")
# Convert 'actual_start_local' to datetime and extract the hour
ping1["actual_start_local"] = pd.to_datetime(ping1["actual_start_local"])
ping1["hour"] = ping1["actual_start_local"].dt.hour - 1

# Convert 'time' in air_quality to datetime and extract the hour
air_quality["time"] = pd.to_datetime(air_quality["UTC"])
air_quality["hour"] = air_quality["time"].dt.hour

# Extract the date from 'actual_start_local'
ping1["date"] = ping1["actual_start_local"].dt.date

# Extract the date from 'UTC' in air_quality
air_quality["date"] = air_quality["time"].dt.date

# Add an additional condition in the loop to match the dates
for index, row in ping1.iterrows():
    op = []  # Initialize an empty list for each bounding box
    for index1, row1 in air_quality.iterrows():
        if (
            isInsideBbox(row1["Latitude"], row1["Longitude"], row["bbox"])
            and row["hour"] == row1["hour"]
            and row["date"] == row1["date"]
        ):
            op.append(
                {
                    "Latitude": row1["Latitude"],
                    "Longitude": row1["Longitude"],
                    "UTC": row1["UTC"],
                    "Parameter": row1["Parameter"],
                    "AQI": row1["AQI"],
                    "Category": row1["Category"],
                }
            )
    results.append(pd.DataFrame(op))

In [54]:
# Initialize a dictionary to store the average PM2.5 and PM10 values
avg_PM25 = {}
avg_PM10 = {}

# Calculate the average PM2.5 and PM10 for each bounding box
for i, df in enumerate(results):
    if not df.empty:
        avg_PM25[i] = df[df["Parameter"] == "PM2.5"]["AQI"].mean()
        avg_PM10[i] = df[df["Parameter"] == "PM10"]["AQI"].mean()
    else:
        avg_PM25[i] = None
        avg_PM10[i] = None

# Add the average PM2.5 and PM10 as new columns to the ping1 DataFrame
ping1["avg_PM25"] = pd.Series(avg_PM25)
ping1["avg_PM10"] = pd.Series(avg_PM10)

# Save the DataFrame to a new CSV file
ping1.to_csv("ping1_avgAQI.csv", index=False)

In [55]:
import pandas as pd
from datetime import datetime, timedelta


def process_ping_data(ping, air_quality_path="air_quality.csv"):
    ping["bbox"] = ping.apply(
        lambda x: latLngToBounds(x["LONGITUDE"], x["LATITUDE"], 12, 400, 400), axis=1
    )

    def isInsideBbox(lat, lng, bbox):
        lng = float(lng)
        lat = float(lat)
        bbox = bbox.split(",")
        if (
            lng >= float(bbox[0])
            and lng <= float(bbox[2])
            and lat >= float(bbox[1])
            and lat <= float(bbox[3])
        ):
            return True
        return False

    results = []

    # Load air quality data
    air_quality = pd.read_csv(air_quality_path)

    # Convert 'actual_start_local' to datetime and extract the hour
    ping["actual_start_local"] = pd.to_datetime(ping["actual_start_local"])
    ping["hour"] = ping["actual_start_local"].dt.hour - 1

    # Convert 'time' in air_quality to datetime and extract the hour
    air_quality["time"] = pd.to_datetime(air_quality["UTC"])
    air_quality["hour"] = air_quality["time"].dt.hour

    # Extract the date from 'actual_start_local'
    ping["date"] = ping["actual_start_local"].dt.date

    # Extract the date from 'UTC' in air_quality
    air_quality["date"] = air_quality["time"].dt.date

    # Filter air quality data inside the bounding box
    for index, row in ping.iterrows():
        op = []  # Initialize an empty list for each bounding box
        for index1, row1 in air_quality.iterrows():
            if (
                isInsideBbox(row1["Latitude"], row1["Longitude"], row["bbox"])
                and row["hour"] == row1["hour"]
                and row["date"] == row1["date"]
            ):
                op.append(
                    {
                        "Latitude": row1["Latitude"],
                        "Longitude": row1["Longitude"],
                        "UTC": row1["UTC"],
                        "Parameter": row1["Parameter"],
                        "AQI": row1["AQI"],
                        "Category": row1["Category"],
                    }
                )
        results.append(pd.DataFrame(op))

    # Initialize a dictionary to store the average PM2.5 and PM10 values
    avg_PM25 = {}
    avg_PM10 = {}

    # Calculate the average PM2.5 and PM10 for each bounding box
    for i, df in enumerate(results):
        if not df.empty:
            avg_PM25[i] = df[df["Parameter"] == "PM2.5"]["AQI"].mean()
            avg_PM10[i] = df[df["Parameter"] == "PM10"]["AQI"].mean()
        else:
            avg_PM25[i] = None
            avg_PM10[i] = None

    # Add the average PM2.5 and PM10 as new columns to the ping DataFrame
    ping["avg_PM25"] = pd.Series(avg_PM25)
    ping["avg_PM10"] = pd.Series(avg_PM10)

    return ping



In [ ]:
# Process each ping DataFrame
for key in dataframes.keys():
    dataframes[key] = process_ping_data(dataframes[key])

# Save the processed DataFrames to CSV files
for key, df in dataframes.items():
    df.to_csv(f"./OutputData/{key}_avgAQI.csv", index=False)